In [ ]:
# !pip install sentence_transformers
# !pip install keras_tuner
# !pip install python-louvain

# from google.colab import drive
# drive.mount('/content/drive')


import csv
import json
import re
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import normalized_mutual_info_score
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import NearestCentroid
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score,pairwise_distances
from sklearn.preprocessing import normalize
from imblearn.under_sampling import AllKNN
from imblearn.under_sampling import RandomUnderSampler
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import plotly.graph_objects as go
import plotly.express as px
import networkx as nx
import networkx.algorithms.community as nx_comm
from community import community_louvain
from ast import literal_eval
np.set_printoptions(linewidth=25000)

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/TCC/df_train_new.txt')
df_test = pd.read_csv('/content/drive/MyDrive/TCC/df_test_new.txt')

def limpa(x):
    return [float(n) for n in (x.strip('[]').replace(' ',',').replace(',,',',').split(',')) if n != '' ]

df_train['Embedding'] = df_train['Embedding'].apply(lambda x: limpa(x))
df_test['Embedding'] = df_test['Embedding'].apply(lambda x: limpa(x))

def limpa2(x):
    return int(x.strip('[]'))

df_train['Genre_Class'] = df_train['Genre_Class'].apply(lambda x: limpa2(x))
df_test['Genre_Class'] = df_test['Genre_Class'].apply(lambda x: limpa2(x))

print(df_train['Genre_Class'].value_counts())
print(df_test['Genre_Class'].value_counts())

def lista(x):
    return [x]

ohe = OneHotEncoder(sparse=False)
ohe_train = pd.Series(list(ohe.fit_transform(df_train['Genre_Class'].apply(lambda x: lista(x)).tolist())))
ohe_test = pd.Series(list(ohe.fit_transform(df_test['Genre_Class'].apply(lambda x: lista(x)).tolist())))

df_train['ohe_train'] = ohe_train
df_test['ohe_test'] = ohe_test

df_train.info()
df_test.info()

plt.hist(df_train['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title("Distribuição da quantidade de dados de cada gênero\nConjunto de Treino")
plt.show()

plt.hist(df_test['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title('Distribuição da quantidade de dados de cada gênero\nConjunto de Teste')
plt.show()


In [ ]:
rus = RandomUnderSampler(random_state=666)
df_train_rus,_ = rus.fit_resample(df_train,df_train['Genre_Class'].tolist())
print(df_train_rus['Genre_Class'].value_counts())

df_train = df_train_rus.copy()

plt.hist(df_train_rus['Genre_Class'],[-0.5,0.5,1.5,2.5,3.5,4.5],rwidth=0.85)
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.title('Distribuição da quantidade de dados de cada gênero\nConjunto de Treino após Subsampling')
plt.show()

In [ ]:
#acc = []
#for n in range(1,40):
n = 13
train_classifier = KNeighborsClassifier(n_neighbors = n, metric = 'euclidean')
train_classifier.fit(np.array(df_train['Embedding'].tolist()), np.array(df_train['Genre_Class'].tolist()))

y_pred = train_classifier.predict(np.array(df_test['Embedding'].tolist()))


cm = confusion_matrix(np.array(df_test['Genre_Class'].tolist()), y_pred)
ac = accuracy_score(np.array(df_test['Genre_Class'].tolist()),y_pred)
#    acc.append([n,ac])



print(cm)
print(ac)
#or i in acc:
#    print(i)

In [ ]:
rus_test = RandomUnderSampler(random_state=666)
df_test_rus,_ = rus_test.fit_resample(df_test,df_test['Genre_Class'].tolist())

print(df_test_rus['Genre_Class'].value_counts())

y_pred = train_classifier.predict(np.array(df_test_rus['Embedding'].tolist()))


cm = confusion_matrix(np.array(df_test_rus['Genre_Class'].tolist()), y_pred)
ac = accuracy_score(np.array(df_test_rus['Genre_Class'].tolist()),y_pred)

print(cm)
print(ac)

print(precision_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))
print(recall_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))
print(f1_score(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, average=None))

disp = ConfusionMatrixDisplay.from_predictions(np.array(df_test_rus['Genre_Class'].tolist()), y_pred, display_labels=selected_genres, xticks_rotation=60, cmap="gist_yarg")

In [ ]:
modelNN = keras.models.load_model('/content/drive/MyDrive/TCC/modelo-NN')

score = modelNN.evaluate(np.array(df_test['Embedding'].tolist()), np.array(df_test['ohe_test'].tolist()))
print('---------------')
print("Perda (entropia cruzada): ", score[0])
print("Categorical accuracy: ", score[1])
print("Falsos negativos: ", score[2])
print("Falsos positivos: ", score[3])
print("Verdadeiros negativos: ", score[4])
print("Verdadeiros positivos: ", score[5])
print("Precisao: ", score[6])
print("Revocacao: ", score[7])

In [ ]:


pred = modelNN.predict(np.array(df_test_rus['Embedding'].tolist()))

print(np.argmax(pred, axis=1))

cm_NN = confusion_matrix(np.array(df_test_rus['Genre_Class'].tolist()), np.argmax(pred, axis=1))
ac_NN = accuracy_score(np.array(df_test_rus['Genre_Class'].tolist()),np.argmax(pred, axis=1))

print(cm_NN)
print(ac_NN)

print(precision_score(np.array(df_test_rus['Genre_Class'].tolist()), np.argmax(pred, axis=1), average=None))
print(recall_score(np.array(df_test_rus['Genre_Class'].tolist()), np.argmax(pred, axis=1), average=None))
print(f1_score(np.array(df_test_rus['Genre_Class'].tolist()), np.argmax(pred, axis=1), average=None))

disp = ConfusionMatrixDisplay.from_predictions(np.array(df_test_rus['Genre_Class'].tolist()), np.argmax(pred, axis=1), display_labels=selected_genres, xticks_rotation=60, cmap="gist_yarg")